In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances    #-} 

### Syntax

In [2]:
data Register = ACC
              | NIL
           -- | ANY
           -- | LAST
              | IN
              | OUT
           -- non-addressable:
              | BAK
              | IP
              deriving (Show, Read, Eq)

data Operand = Reg Register
             | Con Int
             deriving (Read)

type Location = Int

type Condition = Int -> Bool

data Operation = MOV Operand Register
            -- | NOP -> ADD NIL
               | SWP
            -- | SAV -> MOV ACC BAK
               | ADD Operand
               | SUB Operand
               | NEG
               | JMP Condition Location
            -- | JEZ | JNZ | JGZ | JLZ
            -- | JRO Operand
               deriving (Show, Read)

type Node = [Operation]



-- instances

instance Show Operand where
  show (Reg x) = show x
  show (Con x) = show x

instance Show Condition where
  show _ = "c"
  
instance Read Condition where
  readsPrec _ _ = []

### Parsing

In [3]:
import Text.Read
import Data.Maybe
import Control.Monad

import Parsing

In [4]:
word = token $ many upper

register :: Parser Register
register = do xs <- word
              guard $ isJust (readMaybe xs :: Maybe Register)
              return (read xs :: Register)

operand :: Parser Operand
operand =  Reg <$> register
       <|> Con <$> integer

operation :: Parser Operation
operation =  do symbol "NOP" 
                return (ADD (Reg NIL))
         <|> do symbol "SAV"
                return (MOV (Reg ACC) BAK)
         <|> do symbol "MOV"
                o <- operand
                MOV o <$> register
         <|> do symbol "ADD"
                ADD <$> operand
         <|> do symbol "SUB"
                SUB <$> operand
         <|> do symbol "JMP"
                JMP (const True) <$> int
         <|> do symbol "JEZ"
                JMP (== 0)       <$> int
         <|> do symbol "JNZ"
                JMP (/= 0)       <$> int
         <|> do symbol "JGZ"
                JMP (>  0)       <$> int
         <|> do symbol "JLZ"
                JMP (<  0)       <$> int
         <|> do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operation)
                return (read xs :: Operation)
                
node = many operation

parseNode :: String -> Node
parseNode xs = case parse node xs of
                   [(p , [])] -> p
                   [(_ , s)]  -> error ("syntax: unparsed string " ++ s)
                   _          -> error "syntax: failed to parse"

### Interpreting

In [5]:
type State = Register -> Int

init :: State
init _ = 0

update :: Register -> Int -> State -> State
update NIL _ s = s
update r x s = s'
 where s' r' | r' == r   = x
             | otherwise = s r'

eval :: Operand -> State -> Int
eval (Reg r) s = s r
eval (Con x) _ = x

exec :: Operation -> State -> State
exec (MOV o r) s = update r (eval o s) s
exec  SWP      s = update BAK (s ACC) $ update ACC (s BAK) s
exec (ADD o)   s = update ACC (s ACC + eval o s) s
exec (SUB o)   s = update ACC (s ACC - eval o s) s
exec  NEG      s = update ACC (negate (s ACC)) s
exec (JMP c l) s = if c $ s ACC then update IP l s else s

run :: Node -> Int -> Int
run n i = run' n (update IN i init) OUT 

run' :: Node -> State -> State
run' n s = if s IP >= length n then s else
           run' n $ exec (n !! s IP) $ update IP (s IP + 1) s

### Testing

In [6]:
exampleNode = unlines [
              "MOV IN  ACC",
              "SAV        ",
              "MOV 1   ACC",
              "           ",
              "SWP        ", -- line 3
              "JEZ 9      ",
              "SUB 1      ",
              "SWP        ",
              "ADD ACC    ",
              "JMP 3      ",
              "           ",
              "SWP        ", -- line 9
              "MOV ACC OUT"]

putStr exampleNode

MOV IN  ACC
SAV        
MOV 1   ACC
           
SWP        
JEZ 9      
SUB 1      
SWP        
ADD ACC    
JMP 3      
           
SWP        
MOV ACC OUT

In [7]:
parsedNode = parseNode exampleNode
parsedNode

[MOV IN ACC,MOV ACC BAK,MOV 1 ACC,SWP,JMP c 9,SUB 1,SWP,ADD ACC,JMP c 3,SWP,MOV ACC OUT]

In [8]:
map (run parsedNode) [0..9]

[1,2,4,8,16,32,64,128,256,512]